# Analysis of Tailored Forming Manufacturing Processes

## 0. Loading data from ORKG SPAQRL endpoint

In [ ]:
import sparql_dataframe

ENDPOINT_URL = "https://www.orkg.org/orkg/triplestore"

PREFIXES =  """
            PREFIX orkgr: <http://orkg.org/orkg/resource/>
            PREFIX orkgc: <http://orkg.org/orkg/class/>
            PREFIX orkgp: <http://orkg.org/orkg/predicate/>
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            """

query = """
        SELECT ?contrib ?contrib_label ?component_label ?layer_label ?material_name ?order ?step_n_plus_1_name_label ?method_label ?quality_label ?numeric_value ?textual_value (xsd:float(?lower_num_value) AS ?lower_nummeric_value) (xsd:float(?upper_num_value) AS ?upper_numeric_value) ?unit_label
            WHERE {
                #Get all steps and their resulting component (output) and measurement methods (method)
                ?contrib a orkgc:C21027.
                ?contrib orkgp:P7108 ?process_plan;
                        rdfs:label ?contrib_label.
                ?process_plan orkgp:P44026 ?step_n.
                ?step_n orkgp:P7111 ?step_n_name;
                        orkgp:P44025* ?step_n_plus_1;
                        orkgp:P44012 ?method_n;
                        orkgp:P30012 ?output_n.
                ?step_n_plus_1 orkgp:P7111 ?step_n_plus_1_name;
                                orkgp:P44012 ?method_n_plus_1;
                                orkgp:P30012 ?output_n_plus_1.
                BIND(IF(?step_n_plus_1 = ?step_n, ?method_n, ?method_n_plus_1) AS ?method)
                BIND(IF(?step_n_plus_1 = ?step_n, ?output_n, ?output_n_plus_1) AS ?output)
                #Get all qualities from all layers of the resulting component
                ?output orkgp:P28005 ?layer;
                        orkgp:P7111 ?component_type.
                ?component_type rdfs:label ?component_label.
                ?layer orkgp:P3 ?layer_type; #all layers
                        orkgp:P7000 ?material.
                ?layer_type rdfs:label ?layer_label.
                ?material orkgp:P7113 ?quality;
                            orkgp:P42007 ?material_type.
                ?material_type rdfs:label ?material_name.
                ?quality orkgp:P7111 ?quality_type;
                        orkgp:P44022 ?measurement.
                ?quality_type rdfs:label ?quality_label.
                #Get the measurement method for each measurement
                ?method orkgp:P7111 ?method_name;
                        orkgp:HAS_RESULTS ?measurement.
                ?method_name rdfs:label ?method_label.
                # Get the range and unit of the resulting component
                OPTIONAL{?measurement orkgp:P44010 ?range.
                        ?range orkgp:P43248 ?lower_value;
                                orkgp:P43247 ?upper_value.
                        ?upper_value orkgp:P42000 ?range_unit.
                        ?lower_value orkgp:P44011 ?lower_num_value.
                        ?upper_value orkgp:P44011 ?upper_num_value.}
                # Get the measurement value and unit of the resulting component
                OPTIONAL{?measurement orkgp:HAS_VALUE ?value.
                        ?value orkgp:P44011 ?measurement_value.
                        BIND(IF(REGEX(?measurement_value, "^\\d*\\.?\\d*$", "i"), xsd:float(?measurement_value), "") AS ?numeric_value)
                        BIND(IF(REGEX(?measurement_value, "^\\d*\\.?\\d*$", "i"), "", ?measurement_value) AS ?textual_value)
                OPTIONAL{?value orkgp:P42000 ?value_unit.}}
                #If no value is reported, calculate the average value based on the reported range
                #BIND(IF (BOUND(?value), ?measurement_value, (((xsd:int(?lower_numeric_value) + xsd:int(?upper_numeric_value)) / 2) AS ?avg_value) ) AS ?value_calc).
                #Get the unit either from the value or the range
                BIND(IF (BOUND(?value), ?value_unit, ?range_unit) AS ?unit).
                OPTIONAL {?unit rdfs:label ?unit_label.}
                #Get the number of the step in the process plan
                ?step_n_plus_1_name rdfs:label ?step_n_plus_1_name_label.
                BIND(STR(?step_n_plus_1_name_label) AS ?step)
                VALUES(?step ?order){("Deposition welding" 1) ("Hot forming" 2) ("Cooling" 3) ("Heat treatment" 4) ("Machining" 5)}
            }
        GROUP BY ?contrib ?contrib_label ?component_label ?layer_label ?material_name ?order ?step_n_plus_1_name_label ?method_label ?quality_label
        ORDER BY ?contrib ?contrib_label ?component_label ?order ?step_n_plus_1_name_label ?method_label ?quality_label
        """

data = sparql_dataframe.get(ENDPOINT_URL, PREFIXES+query)
data.head()

## 1. Loading data from CSV file

In [10]:
import pandas as pd

tf_data = pd.read_csv('tf-query-2022-04-04.csv')
tf_data.head()

,contrib,contrib_label,component_label,layer_label,material_name,order,step_n_plus_1_name_label,method_label,quality_label,numeric_value,textual_value,lower_nummeric_value,upper_numeric_value,unit_label
0,http://orkg.org/orkg/resource/R145728,1 Cladding Layer Component (100Cr6),Bearing washer,Base material,C22.8,1,Deposition welding,Hardness profile,Hardness,NaN,NaN,100.0,200.0,HV0.5
1,http://orkg.org/orkg/resource/R145728,1 Cladding Layer Component (100Cr6),Bearing washer,Cladding layer,100Cr6,1,Deposition welding,Hardness profile,Hardness,NaN,NaN,350.0,450.0,HV0.5
2,http://orkg.org/orkg/resource/R145728,1 Cladding Layer Component (100Cr6),Bearing washer,Cladding layer,100Cr6,1,Deposition welding,Micrograph,Microstructure,NaN,pearlitic,NaN,NaN,NaN
3,http://orkg.org/orkg/resource/R145728,1 Cladding Layer Component (100Cr6),Bearing washer,Base material,C22.8,1,Deposition welding,Micrograph,Microstructure,NaN,ferritic-pearlitic,NaN,NaN,NaN
4,http://orkg.org/orkg/resource/R145728,1 Cladding Layer Component (100Cr6),Bearing washer,Cladding layer,100Cr6,1,Deposition welding,Nano-scratch test,Indentation depth,10.6,NaN,NaN,NaN,Nanometer


In [24]:
steps = tf_data.sort_values('order')['order'].unique()
print(steps)



[1 2 3 4 5]
